# Rating a Single-Stage Vapor Compression Machine
---

In this notebook we look at another example of rating a single-stage vapor compression machine used for cooling supply air in an air conditioning system. The machine is equipped with a variable speed compressor. In this example we will determine the required compressor speed when the refrigeration machine is operating at design conditions. These design conditions include: 
1. the state and mass flow rate of air entering the evaporator (which are both given from the start),
2. the state and mass flow rate of outdoor air entering the condenser (only the state of outdoor air is known from the start),
3. a selected evaporation temperature of the refrigerant,
4. a selected condensing temperature of the refrigerant, and
5. a required amount of superheat of refrigerant leaving the evaporator.

In [1]:
import warnings
from hvac import Quantity
from hvac.fluids import HumidAir
from hvac.fluids.fluid_experimental import PureFluid
from hvac.fluids.exceptions import CoolPropWarning
from hvac.heat_exchanger.recuperator.fintube import PFT_CF_AE, PFT_CF_AC
from hvac.vapor_compression import VariableSpeedCompressor, SS_VCM

In [2]:
warnings.filterwarnings('ignore', category=CoolPropWarning)

In [3]:
Q_ = Quantity
R410A = PureFluid('R410A')

## Design Conditions

Air with a mass flow rate of `evp_air_m_dot` and with a known state `evp_air_in` enters an air evaporator (DX-coil) and needs to be cooled to a state `evp_air_out`. 

In [4]:
evp_air_in = HumidAir(
    Tdb=Q_(28.2, 'degC'),
    W=Q_(10.6, 'g / kg')
)
evp_air_out = HumidAir(
    Tdb=Q_(14.0, 'degC'),
    W=Q_(9.1, 'g / kg')
)
evp_air_m_dot = Q_(10_762.4, 'kg / hr')

The heat rate that needs to be extracted from the air flow in the evaporator follows from:

In [5]:
evp_Q_dot = evp_air_m_dot * (evp_air_in.h - evp_air_out.h)
print(f"required heat absorption rate = {evp_Q_dot.to('kW'):~P.3f}")

required heat absorption rate = 54.885 kW


We need to select the evaporation temperature of the refrigerant in the evaporator. Let us select an evaporation temperature which is 10 K below the air leaving temperature.

In [6]:
evp_T_rfg = evp_air_out.Tdb - Q_(10, 'K')
print(f"evaporation temperature = {evp_T_rfg.to('degC'):~P.2f}")

evaporation temperature = 4.00 °C


The refrigerant, which was already defined above, is R410A. We can determine the saturation pressure that corresponds with the selected evaporation temperature. 

In [7]:
evp_rfg_sat_vap = R410A(T=evp_T_rfg, x=Q_(1.0, 'frac'))  # saturated vapor
evp_P_rfg = evp_rfg_sat_vap.P
print(f"evaporation pressure = {evp_P_rfg.to('bar'):~P.3f}")

evaporation pressure = 9.049 bar


At the outlet of evaporator the refrigerant needs to be superheated enough to prevent liquid refrigerant from entering the compressor, as this would damage the compressor. We need to select the degree of refrigerant superheating, which will be a setting of the expansion device (TXV). 

In [8]:
dT_sh = Q_(10, 'K')

Once the degree of refrigerant superheating has been selected, we can determine the state of refrigerant leaving the evaporator. 

In [9]:
evp_rfg_out = R410A(P=evp_P_rfg, T=evp_T_rfg + dT_sh)

The state of refrigerant entering the evaporator will depend on the state of refrigerant leaving the condenser. In the expansion device the refrigerant undergoes an isenthalpic expansion process, which means that the enthalpy of refrigerant entering the evaporator is the same as the enthalpy of refrigerant leaving the condenser. This means that we first need to determine the design conditions at the condenser.

At the condenser outdoor air enters the condenser with a known state `cnd_air_in`.

In [10]:
cnd_air_in = HumidAir(Tdb=Q_(32, 'degC'), Twb=Q_(21, 'degC'))

When outdoor air flows through the condenser it takes up heat from the condensing refrigerant. We can select the allowable temperature rise of the air flow and determine the state of air leaving the condenser, knowing that air heating is a sensible-heat-only process (i.e., the absolute humidity of the air stays constant).

In [11]:
cnd_dT_air = Q_(10, 'K')
cnd_air_out = HumidAir(Tdb=cnd_air_in.Tdb + cnd_dT_air, W=cnd_air_in.W)
print(f"temperature of air leaving condenser = {cnd_air_out.Tdb.to('degC'):~P.1f}")

temperature of air leaving condenser = 42.0 °C


Now we can select the refrigerant condensing temperature. Lets take it 10 K above the temperature of air leaving the condenser.

In [12]:
cnd_T_rfg = cnd_air_out.Tdb + Q_(10, 'K')
print(f"condensing temperature = {cnd_T_rfg.to('degC'):~P.2f}")

condensing temperature = 52.00 °C


The corresponding saturation pressure of the refrigerant follows from:

In [13]:
cnd_rfg_sat_liq = R410A(T=cnd_T_rfg, x=Q_(0.0, 'frac'))  # saturated liquid
cnd_P_rfg = cnd_rfg_sat_liq.P
print(f"condensing pressure = {cnd_P_rfg.to('bar'):~P.3f}")

condensing pressure = 32.147 bar


To ensure that liquid refrigerant will enter the expansion device, the refrigerant leaving the condenser should be subcooled to a certain degree:

In [14]:
dT_sc = Q_(10, 'K')

Once the degree of refrigerant subcooling has been selected, we can determine the state of refrigerant leaving the condenser.

In [15]:
cnd_rfg_out = R410A(P=cnd_P_rfg, T=cnd_T_rfg - dT_sc)
print(f"temperature of refrigerant leaving condenser = {cnd_rfg_out.T.to('degC'):~P.2f}")
print(f"enthalpy of refrigerant leaving condenser = {cnd_rfg_out.h.to('kJ / kg'):~P.3f}")

temperature of refrigerant leaving condenser = 42.00 °C
enthalpy of refrigerant leaving condenser = 269.144 kJ/kg


Now that we have determined the state of refrigerant leaving the condenser, we can go back to the design of the evaporator and determine the state of refrigerant entering the evaporator, assuming that the enthalpy of the refrigerant remains constant between the condenser's exit and the evaporator's entrance.

In [16]:
evp_rfg_in = R410A(P=evp_P_rfg, h=cnd_rfg_out.h)
print(f"vapor quality of refrigerant entering evaporator = {evp_rfg_in.x.to('frac'):~P.2f}")

vapor quality of refrigerant entering evaporator = 0.29 frac


We know the states of refrigerant entering and leaving the evaporator and we know the heat rate that must be absorbed by the refrigerant in the evaporator. This allows us to determine the required mass flow rate of refrigerant as follows: 

In [17]:
rfg_m_dot = evp_Q_dot / (evp_rfg_out.h - evp_rfg_in.h)
print(f"required mass flow rate of refrigerant = {rfg_m_dot.to('kg / h'):~P.3f}")

required mass flow rate of refrigerant = 1201.076 kg/h


## Air Evaporator Design

To design the air evaporator, we need to determine the width and height of the air-side frontal area, choose the geometrical properties of the heat exchanger core, and select the number of tube rows. To determine the size of the frontal area, we start with selecting the face air velocity.

In [18]:
evp_v_fa = Q_(2, 'm / s')

Given the air mass flow rate through the evaporator and knowing the state of air entering the evaporator, the air volume flow rate at the entrance of the evaporator can be determined.  

In [19]:
evp_air_V_dot = evp_air_m_dot / evp_air_in.rho
print(f"air volume flow rate at evaporator entrance = {evp_air_V_dot.to('m**3 / h'):~P.3f}") 

air volume flow rate at evaporator entrance = 9341.150 m³/h


Based on the selected face air velocity, the air-side frontal area of the evaporator can be determined. 

In [20]:
evp_A_fa = evp_air_V_dot / evp_v_fa
print(f"air-side frontal area = {evp_A_fa.to('m ** 2'):~P.3f}")

air-side frontal area = 1.297 m²


To determine the width and height of the air-side frontal area, we select an aspect ratio (ratio of height to width).

In [21]:
evp_aspect_ratio = 1 / 3
evp_width = (evp_A_fa / evp_aspect_ratio) ** 0.5
evp_height = evp_aspect_ratio * evp_width
print(
    f"frontal area width = {evp_width.to('mm'):~P.0f}",
    f"frontal area height = {evp_height.to('mm'):~P.0f}",
    sep='\n'
)

frontal area width = 1973 mm
frontal area height = 658 mm


We will design a plain fin-tube, counter-flow air evaporator for which we choose an initial number of rows.

In [22]:
Evaporator = PFT_CF_AE  # Plain Fin-Tube Counter-Flow Air Evaporator

In [23]:
evaporator = Evaporator(
    W_fro=evp_width,              # width of frontal area
    H_fro=evp_height,             # height of frontal area
    N_rows=3,                     # number of rows (initial guess)
    S_trv=Q_(22.42, 'mm'),        # vertical distance between tubes
    S_lon=Q_(22.27, 'mm'),        # horizontal distance between tubes
    D_int=Q_(8.422, 'mm'),        # inner tube diameter
    D_ext=Q_(10.2, 'mm'),         # outer tube diameter
    t_fin=Q_(0.3302, 'mm'),       # fin thickness
    N_fin=1 / Q_(3.2, 'mm'),      # fin density
    k_fin=Q_(237, 'W / (m * K)')  # conductivity of fin material (here: aluminium)
)

To find the actual needed number of rows, we "solve" the instantiated evaporator model. The solving routine requires the state of air entering the evaporator, the air mass flow rate through the evaporator, the state of refrigerant entering the evaporator, and the required degree of refrigerant superheating at the exit of the evaporator. Based on this design data, the evaporator model will calculate the refrigerant mass flow rate the expansion device lets through so that the refrigerant is superheated to the required degree. 

In [24]:
rfg_m_dot_new = evaporator.solve(
    air_in=evp_air_in,
    air_m_dot=evp_air_m_dot,
    rfg_in=evp_rfg_in,
    dT_sh=dT_sh,
)

In [25]:
print(f"refrigerant mass flow rate = {rfg_m_dot_new.to('kg / h'):~P.3f}")

refrigerant mass flow rate = 1287.142 kg/h


If the refrigerant mass flow rate `rfg_m_dot_new` needed to superheat the refrigerant to the required degree `dT_sh` is (much) more than the refrigerant mass flow rate `rfg_m_dot` previously calculated, it indicates that the evaporator will be oversized. In that case, we should decrease the number of rows `N_rows`. Conversely, if `rfg_m_dot_new` is (much) less than `rfg_m_dot`, we should increase the number of rows `N_rows`. Also, be changing fin density (fin spacing) `N_fin`, we can further minimize the difference between `rfg_m_dot_new` and `rfg_m_dot`.

## Compressor Selection

We use the selection software COOLSELECTOR2 from DANFOSS to select a compressor with a variable speed drive. To select a suitable compressor, we need to give the program the refrigerant that we are planning to use, the cooling capacity that we need, the selected evaporation and condensing temperature of the refrigerant, and the desired degree of refrigerant superheating and subcooling. According to the program, the hermetic scroll compressor VZH117CGM, driven at speed of 5560 rpm, is our best candidate.

To model this compressor in our notebook, we need to export the coefficients of the polynomial equations, which determine the performance of the compressor as a function of evaporation temperature, condensing temperature, and compressor speed, as a spreadsheet file to our computer. After this file has been saved on our computer, we can prepare the csv-file that is needed to instantiate the compressor model in this notebook. This csv-file is saved as `compressor_VZH117CGM.csv` in the same directory as our notebook. Now, we can create an instance of the compressor model. 

In [26]:
compressor = VariableSpeedCompressor(
    coeff_file="./compressor_data/VZH117CGM_R410a.csv",
    refrigerant=R410A,
    dT_sh=dT_sh,
    dT_sc=dT_sc,
    units={'n': 'rps'}
)

## Air Condenser Design

In the condenser, the refrigerant flow must reject its heat to the outdoor air flow in order to leave the condenser as a subcooled liquid so that the refrigeration cycle can be repeated. The heat rate that must be rejected in the condenser is the sum of the heat rate the refrigerant absorbed in the evaporator and the mechanical power the refrigerant received from the compressor. We have already determined the heat rate absorbed by the refrigerant in the evaporator. To determine the mechanical power that the compressor added to the refrigerant, we can use our compressor model. For this, we first need to set the operating state of the compressor. This operating state is determined by the evaporation temperature, the condensing temperature, and the speed at which the compressor is driven. For the compressor speed, we take the value indicated by the COOLSELCTOR2 selection software when we selected the compressor.

In [27]:
compressor.T_evp = evp_T_rfg
compressor.T_cnd = cnd_T_rfg
compressor.speed = Q_(5560, 'rpm')

Now, we can easily retrieve the mechanical power the compressor adds to the refrigerant when operating under design conditions. 

In [28]:
print(f"compressor power = {compressor.W_dot.to('kW'):~P.3f}")

compressor power = 17.903 kW


The heat rate that must be rejected by the refrigerant in the condenser is then:

In [29]:
cnd_Q_dot = evaporator.Q_dot + compressor.W_dot
print(f"required heat rejection rate = {cnd_Q_dot.to('kW'):~P.3f}")

required heat rejection rate = 76.751 kW


In the first stage of our design calculations, we have already determined the states of air entering and leaving the condenser. Now, we can also determine the required mass flow rate of outdoor air through the condenser.

In [30]:
cnd_air_m_dot = cnd_Q_dot / (cnd_air_out.h - cnd_air_in.h)
print(f"required air mass flow rate through condenser = {cnd_air_m_dot.to('kg / h'):~P.3f}")

required air mass flow rate through condenser = 26885.303 kg/h


In the same way as with the evaporator, we can size the air-side frontal area of the condenser by selected an allowable face air velocity.

In [31]:
cnd_v_fa = Q_(2, 'm / s')
cnd_air_V_dot = cnd_air_m_dot / cnd_air_in.rho
print(f"air volume flow rate at condenser entrance = {cnd_air_V_dot.to('m**3 / h'):~P.3f}")

air volume flow rate at condenser entrance = 23648.981 m³/h


In [32]:
cnd_A_fa = cnd_air_V_dot / cnd_v_fa
print(f"air-side frontal area = {cnd_A_fa.to('m ** 2'):~P.3f}")

air-side frontal area = 3.285 m²


In [33]:
cnd_aspect_ratio = 1 / 3
cnd_width = (cnd_A_fa / cnd_aspect_ratio) ** 0.5
cnd_height = cnd_aspect_ratio * cnd_width
print(
    f"frontal area width = {cnd_width.to('mm'):~P.0f}",
    f"frontal area height = {cnd_height.to('mm'):~P.0f}",
    sep='\n'
)

frontal area width = 3139 mm
frontal area height = 1046 mm


Before we can instantiate the air condenser model, there's still one thing we need, which is the state of refrigerant entering the condenser. If we assume that the refrigerant state remains unchanged between the exit of the compressor and the entrance of the condenser, we can use the state of the discharge gas from our compressor model.   

In [34]:
cnd_rfg_in = compressor.discharge_gas
print(f"temperature of refrigerant entering the condenser = {cnd_rfg_in.T.to('degC'):~P.2f}")

temperature of refrigerant entering the condenser = 94.49 °C


Like the air evaporator, we will model the air condenser as a plain fin-tube, counter-flow heat exchanger.

In [35]:
Condenser = PFT_CF_AC

In [36]:
condenser = Condenser(
    W_fro=cnd_width,              # width of frontal area
    H_fro=cnd_height,             # height of frontal area
    N_rows=3,                     # number of rows (initial guess)
    S_trv=Q_(22.42, 'mm'),        # vertical distance between tubes
    S_lon=Q_(22.27, 'mm'),        # horizontal distance between tubes
    D_int=Q_(8.422, 'mm'),        # inner tube diameter
    D_ext=Q_(10.2, 'mm'),         # outer tube diameter
    t_fin=Q_(0.3302, 'mm'),       # fin thickness
    N_fin=1 / Q_(3.2, 'mm'),      # fin density
    k_fin=Q_(237, 'W / (m * K)')  # conductivity of fin material (here, aluminium)
)

When we want to "solve" the air condenser model, we need to pass the known operating parameters to the solving routine. On the air-side of the condenser these are the state of air entering the condenser and the air mass flow rate. On the refrigerant-side of the condenser these are the state of refrigerant entering the condenser and the refrigerant mass flow rate. Solving the condenser model means that the still unknown states of air and refrigerant leaving the condenser are calculated, while the geometrical properties and the size of the heat exchanger core are fixed. Should the solving routine raise an exception, it probably means that the number of rows is too small to fully condense and subcool the refrigerant. It this happens, we can increment the value of parameter `N_rows` until the solving routine returns without raising an exception (here in this case a `CondenserError` is raised if the number of rows is smaller than 3). By modifying the fin density `N_fin` it is also possible to control the degree of refrigerant subcooling. 

In [37]:
cnd_air_out_new, cnd_rfg_out_new = condenser.solve(
    air_in=cnd_air_in,
    air_m_dot=cnd_air_m_dot,
    rfg_in=cnd_rfg_in,
    rfg_m_dot=rfg_m_dot
)

From the solving routine we get a new state for the refrigerant leaving the condenser. If evaporator, compressor, and condenser are perfectly matched, this new state should be identical to the state we previously used for determining the state of refrigerant entering the evaporator, otherwise the refrigeration cycle would not be repeatable.   

In [38]:
print(f"temperature of refrigerant leaving condenser = {cnd_rfg_out_new.T.to('degC'):~P.2f}")
print(f"enthalpy of refrigerant leaving condenser = {cnd_rfg_out_new.h.to('kJ / kg'):~P.3f}")
print(f"subcooling = {condenser.dT_sc:~P.2f}")

temperature of refrigerant leaving condenser = 39.26 °C
enthalpy of refrigerant leaving condenser = 264.001 kJ/kg
subcooling = 12.63 K


We started our calculations with:

In [39]:
print(f"temperature of refrigerant leaving condenser = {cnd_rfg_out.T.to('degC'):~P.2f}")
print(f"enthalpy of refrigerant leaving condenser = {cnd_rfg_out.h.to('kJ / kg'):~P.3f}")

temperature of refrigerant leaving condenser = 42.00 °C
enthalpy of refrigerant leaving condenser = 269.144 kJ/kg


We see that the new state does not coincide with the state we have started with. This means that the actual steady-state performance of our vapor compression machine will be different than what we intended. To balance the vapor compression machine, we can look for the compressor speed for which the selected evaporation temperature and condensing temperature will be achieved under the given external operating conditions (these are the state of air entering the evaporator and the condenser, and the air mass flow rate through evaporator and condenser).

## Balancing the Refrigeration Machine

The machine is a single-stage vapor compression machine (i.e. having only one stage of refrigerant compression). 

In [40]:
Machine = SS_VCM  # Single-Stage Vapor Compression Machine

To compose the machine, we use the evaporator, the compressor, and the condenser we designed before. To instantiate the machine model, we also need to specify the degree of refrigerant superheating set on the expansion device, and the minimum and maximum speed of the compressor.

In [41]:
machine = Machine(
    evaporator, condenser, compressor,
    dT_sh,
    n_cmp_min=Q_(1500, 'rpm'),
    n_cmp_max=Q_(6000, 'rpm')
)

To balance the machine, we call the method `balance_by_speed` which will return an `Output` object containing the machine performance data under set external operating conditions and for the selected evaporation and condensing temperature of the refrigerant in the circuit.

In [42]:
output = machine.balance_by_speed(
    evp_air_in=evp_air_in,
    evp_air_m_dot=evp_air_m_dot,
    cnd_air_in=cnd_air_in,
    cnd_air_m_dot=cnd_air_m_dot,
    T_evp=evp_T_rfg,
    T_cnd=cnd_T_rfg
)

[31620 | hvac.vapor_compression.machine | INFO] Starting speed balancing with...
[31620 | hvac.vapor_compression.machine | INFO] evp_air_in: 28.20 °C, 44 %
[31620 | hvac.vapor_compression.machine | INFO] evp_air_m_dot: 10762.40 kg/h
[31620 | hvac.vapor_compression.machine | INFO] cnd_air_in: 32.00 °C, 37 %
[31620 | hvac.vapor_compression.machine | INFO] cnd_air_m_dot: 26885.30 kg/h
[31620 | hvac.vapor_compression.machine | INFO] T_evp = 277.15 K
[31620 | hvac.vapor_compression.machine | INFO] T_cnd = 325.15 K
[31620 | hvac.vapor_compression.machine | INFO] Iteration 1: Try with: 3219.000 rpm
[31620 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant mass flow rate from compressor = 700.104 kg/h
[31620 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant entering condenser with T = 93.452 °C, P = 32.065 bar
[31620 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant leaving condenser with T = 33.753 °C, P = 32.065 bar
[31620 | hvac.vapor_compressi

In [43]:
print(output.to_text())

evp_air_m_dot = 10762.400 kg/h
cnd_air_m_dot = 26885.303 kg/h
evp_air_in = 28.200 °C DB, 10.600 g/kg AH (44 % RH)
cnd_air_in = 32.000 °C DB, 11.111 g/kg AH (37 % RH)
n_cmp = 5831.000 rpm
dT_sh = 10.000 K
evp_air_out = 14.869 °C DB, 8.244 g/kg AH (78 % RH)
cnd_air_out = 42.107 °C DB, 11.111 g/kg AH (21 % RH)
evp_Q_dot = 58.692 kW
cnd_Q_dot = 77.513 kW
cmp_W_dot = 18.858 kW
rfg_m_dot = 1269.939 kg/h
COP = 4.110 frac
EER = 3.112 frac
evp_eps = 0.4823 frac
cnd_eps = 0.4088 frac
T_evp = 4.000 °C
P_evp = 9.049 bar
T_cnd = 51.888 °C
P_cnd = 32.065 bar
dT_sc = 10.823 K
suction_gas = 14.000 °C, 433.651 kJ/kg, 9.049 bar
discharge_gas = 94.712 °C, 487.108 kJ/kg, 32.065 bar
liquid = 41.065 °C, 267.375 kJ/kg, 32.065 bar
mixture = 3.925 °C, 267.375 kJ/kg, 9.049 bar, 28 %
evp_air_dP = 32.749 Pa
cnd_air_dP = 36.484 Pa

